In [7]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import json

In [8]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/mikenlee/.wdm/drivers/chromedriver/mac64/88.0.4324.96]


In [9]:
url = "https://www.fairfaxcounty.gov/transparencyresults/"

In [10]:
browser.visit(url)

### Get list of Fiscal Years to Loop through

In [11]:
#get fiscal year splinter element
fy_element = browser.find_by_name('ctl00$cphMainContent$ddlFy')

#get list of splinter elements for each Fiscal Year Option
fy_options = [x for x in fy_element.find_by_tag("option")]

#get list of actual Fiscal Years
fy_list = [fy.value for fy in fy_options]

fy_list

['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

### Define a function that selects Fiscal Year and two more buttons

In [12]:
def fiscalyear_select(fy):
    #select FY from the drop down
    browser.find_by_name('ctl00$cphMainContent$ddlFy').first.select(fy)

    #select "Budget vs Actual Expenditures"
    browser.find_by_name('ctl00$cphMainContent$btnFundDept').click()

    #select "General Fund"
    fund_element = browser.find_by_name('ctl00$cphMainContent$ddlFund')
    fund_element.select('GENERALFUND')

### Define a function that creates a department list

In [13]:
def department_value_list():    
    #get department splinter element
    dep_element = browser.find_by_name('ctl00$cphMainContent$ddlGeneralFund')
    
    #fetch department NAMES into a list
    dep_names = [x.text for x in dep_element.find_by_tag("option")]
    dep_names.pop(0) #remove the first value since it's not a department

    #fetch department VALUES into a list
    dep_values = [x.value for x in dep_element.find_by_tag("option")]
    dep_values.pop(0) #remove the first value since it's not a department
    
    #create department dictionary as a function attribute so it can be referenced outside of the function
    department_value_list.dep_dict = dict(zip(dep_values, dep_names))


### Define a function that scrapes the table

In [14]:
def department_table_scrape(dep):

    dep_element = browser.find_by_name('ctl00$cphMainContent$ddlGeneralFund')

    #select Department from the drop down options
    dep_element.select(dep)

    #click "View Fund Data" link for table
    browser.find_by_name('ctl00$cphMainContent$btnViewFundData').click()

    #get total expenditures for the department from the HTML table
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    tds = soup.find("tr", class_ = "footer").find_all('td')

    #setdefault initializes the value for the key if that key is not defined, o/w it does nothing
    exp_dict.setdefault('county', []).append("Fairfax")
    exp_dict.setdefault('fiscal_year', []).append(fy)
    exp_dict.setdefault('department', []).append(dep_name)
    exp_dict.setdefault('option_value', []).append(dep)
    exp_dict.setdefault('revised_annual_budget', []).append(tds[1].get_text())
    exp_dict.setdefault('expenditures_ytd', []).append(tds[2].get_text())

### Use above functions to loop through each FY and Department to scrape tables

In [15]:
#intialize expenditure dictionary
exp_dict = {}

#loop through each Fiscal Year
for fy in fy_list:   
    
    try:
        fiscalyear_select(fy)
        
    except:
        #try multiple attempts in case of stale element reference
        attempts = 1
        while (attempts < 4):
            try:
                fiscalyear_select(fy)
                print("FY SUCCESS!")
                break
                
            except Exception as e:
                print(f"{fy}: {e}")
                print(f"Attempt #{attempts}")
                print("---------- at FY")
                attempts += 1

    #####--------------- Get LIST OF DEPARTMENTS to loop through
    
    try:
        department_value_list()

    except:
        attempts = 1
        while (attempts < 4):
            try:
                department_value_list()
                print("Dep SUCCESS!")
                break
                
            except Exception as e:
                print(f"{fy}: {e}")
                print(f"Attempt #{attempts}")
                print("------------ at dep_list")
                attempts += 1

    #####---------------- Loop through each Department value and scrape expenditures table

    for dep, dep_name in department_value_list.dep_dict.items():
        try:
            department_table_scrape(dep)
            
        except Exception as e1:
            print(f"{fy}: {dep_name} -> {dep}")
            print(e1)
            print("------------FIRST exception")
            attempts = 1
            while (attempts < 4):                
                try:
                    department_table_scrape(dep)
                    print("Table SUCCESS!")
                    break
                    
                except Exception as e:
                    print(f"{fy}: {dep_name} -> {dep}")
                    print(e)
                    print(f"Attempt #{attempts}")
                    print("------------")
                    attempts += 1

2017: UNCLASSIFIED ADMINISTRATIVE EXPENSES (PU -> G2587
Message: stale element reference: element is not attached to the page document
  (Session info: chrome=88.0.4324.146)

------------FIRST exception
Table SUCCESS!
2021: FIRE AND RESCUE -> G9292
Message: stale element reference: element is not attached to the page document
  (Session info: chrome=88.0.4324.146)

------------FIRST exception
Table SUCCESS!


In [20]:
expenditureDF = pd.DataFrame(exp_dict)

In [24]:
#remove the 'option_value', 'revised_annual_budget' columns
del expenditureDF['option_value']
del expenditureDF['revised_annual_budget']

,county,fiscal_year,department,expenditures_ytd
0,Fairfax,2013,ADMINISTRATION FOR HUMAN SERVICES,"$11,569,374.71"
1,Fairfax,2013,ALCOHOL AND DRUG SERVICES,$0.00
2,Fairfax,2013,BOARD OF SUPERVISORS,"$4,554,678.86"
3,Fairfax,2013,BUSINESS PLANNING AND SUPPORT,"$739,969.50"
4,Fairfax,2013,CABLE AND CONSUMER SERVICES,"$1,712,730.11"
...,...,...,...,...
450,Fairfax,2021,TAX ADMINISTRATION,"$13,590,873.37"
451,Fairfax,2021,TRANSPORTATION,"$3,735,409.29"
452,Fairfax,2021,UNCLASSIFIED ADMINISTRATIVE EXPENSES (NO,"$26,452,092.17"
453,Fairfax,2021,UNCLASSIFIED ADMINISTRATIVE EXPENSES (PU,"$303,598.00"


In [27]:
#save a csv to load into AWS
expenditureDF.to_csv("/Users/mikenlee/Documents/Personal/analytics/GWU/county_expenditures/static/data/fairfax_expenditures.csv", index = False)

In [11]:
#save a csv version
pd.DataFrame(exp_dict).to_csv("/Users/mikenlee/Documents/Personal/analytics/GWU/county_expenditures/fairfax.csv")

In [43]:
pd.DataFrame(pd.DataFrame(exp_dict)['department'].unique()).to_csv('/Users/mikenlee/Documents/fairfax_departments.csv')

In [41]:
#change format of dictionary
expendituresDF = pd.DataFrame(exp_dict)
exp_dict2 = expendituresDF.to_dict(orient="records")

In [5]:
#write dictionary to a json file
file_location = '/Users/mikenlee/Documents/Personal/analytics/GWU/county_expenditures/static/data/fairfax_data.json'

json.dump(exp_dict2, open(file_location, 'w'))

In [ ]:
#get json format into javascript object format
#this code snippet is used in flask app.py
with open (file_location) as json_file:
    fairfax_data = json.load(json_file)
   
fairfax_data

#### TEST

In [14]:
expendituresDF = pd.DataFrame(exp_dict)
expendituresDF.loc[expendituresDF['fiscal_year'] == "2018"]

,county,fiscal_year,department,option_value,revised_annual_budget,expenditures_ytd
250,Fairfax,2018,ACUTE & THERAPEUTIC TREATMENT,G7620,$0.00,$0.00
251,Fairfax,2018,ADMINISTRATION FOR HUMAN SERVICES,G6868,"$13,899,489.27","$13,421,349.31"
252,Fairfax,2018,ANIMAL SHELTERING,G9696,"$2,518,459.78","$2,161,125.65"
253,Fairfax,2018,BOARD OF SUPERVISORS,G0101,"$5,966,713.40","$5,088,578.08"
254,Fairfax,2018,BUSINESS PLANNING AND SUPPORT,G2525,"$1,110,398.79","$1,009,630.88"
255,Fairfax,2018,CABLE AND CONSUMER SERVICES,G0404,"$831,288.00","$809,284.00"
256,Fairfax,2018,CAPITAL FACILITIES,G2526,"$14,443,984.66","$13,972,068.87"
257,Fairfax,2018,CIRCUIT COURT AND RECORDS,G8080,"$11,448,411.96","$11,396,333.68"
258,Fairfax,2018,CIVIL SERVICE COMMISSION,G4141,"$469,846.00","$452,271.99"
259,Fairfax,2018,CODE COMPLIANCE,G9797,"$4,462,825.52","$4,322,854.86"
